<a href="https://colab.research.google.com/github/OllyMuh/Python_for_DataScience/blob/Project-Clean/Dataset_Cleaning_RealEstate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Анализ данных

In [ ]:
import pandas as pd
import numpy as np

from scipy.stats import mode

# отключаем предупреждения
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# пути к директориям и файлам
DATASET_PATH = '/content/drive/MyDrive/train.csv'
PREPARED_DATASET_PATH = '/content/drive/MyDrive/train_prepared.csv'

**Описание датасета:**

**Id** - идентификационный номер квартиры

**DistrictId** - идентификационный номер района

**Rooms** - количество комнат

**Square** - площадь

**LifeSquare** - жилая 

**KitchenSquare** - площадь кухни

**Floor** - этаж

**HouseFloor** - количество этажей в доме

**HouseYear** - год постройки дома

**Ecology_1, Ecology_2, Ecology_3** - экологические показатели местности

**Social_1, Social_2, Social_3** - социальные показатели 

**Healthcare_1, Helthcare_2** - показатели местности, связанные с охраной здоровья

**Shops_1, Shops_2** - показатели, связанные с наличием магазинов, торговых центров

**Price** - цена квартиры

In [ ]:
df = pd.read_csv(DATASET_PATH, sep=',')
df.head(2)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,11809,27,3.0,115.027311,NaN,10.0,4,10.0,2014,0.075424,B,B,11,3097,0,NaN,0,0,B,305018.871089
1,3013,22,1.0,39.832524,23.169223,8.0,7,8.0,1966,0.118537,B,B,30,6207,1,1183.0,1,0,B,177734.553407


In [ ]:
df.tail(3)

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
9997,2795,178,1.0,29.648057,16.555363,5.0,3,5.0,1958,0.460556,B,B,20,4386,14,NaN,1,5,B,165953.912580
9998,14561,21,1.0,32.330292,22.326870,5.0,3,9.0,1969,0.194489,B,B,47,8004,3,125.0,3,5,B,171842.411855
9999,7202,94,1.0,35.815476,22.301367,6.0,9,9.0,1975,0.127376,B,B,43,8429,3,NaN,3,9,B,177685.627486


In [ ]:
df.dtypes

Id                 int64
DistrictId         int64
Rooms            float64
Square           float64
LifeSquare       float64
KitchenSquare    float64
Floor              int64
HouseFloor       float64
HouseYear          int64
Ecology_1        float64
Ecology_2         object
Ecology_3         object
Social_1           int64
Social_2           int64
Social_3           int64
Healthcare_1     float64
Helthcare_2        int64
Shops_1            int64
Shops_2           object
Price            float64
dtype: object

In [ ]:
# изменяем тип данных id на строковый - чтобы не мешал
df['Id'] = df['Id'].astype(str)
df['Id'].dtype

dtype('O')

In [ ]:
# получаем обобщенную информацию о датасете
df.describe()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  object 
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     7887 non-null   float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Ecology_2      10000 non-null  object 
 11  Ecology_3      10000 non-null  object 
 12  Social_1       10000 non-null  int64  
 13  Social_2       10000 non-null  int64  
 14  Social_3       10000 non-null  int64  
 15  Healthcare_1   5202 non-null   float64
 16  Helthcare_2    10000 non-null  int64  
 17  Shops_1        10000 non-null  int64  
 18  Shops_2

In [ ]:
df.select_dtypes(include='object')

,Id,Ecology_2,Ecology_3,Shops_2
0,11809,B,B,B
1,3013,B,B,B
2,8215,B,B,B
3,2352,B,B,B
4,13866,B,B,B
...,...,...,...,...
9995,1260,B,B,B
9996,16265,B,B,A
9997,2795,B,B,B
9998,14561,B,B,B


In [ ]:
df['Shops_2'].value_counts()
df['Ecology_2'].unique()

array(['B', 'A'], dtype=object)

# Обработка пропусков

In [ ]:
# булевая маска таблицы - пропуски df.isna()
# суммирование по столбцам
df.isna().sum()

Id                  0
DistrictId          0
Rooms               0
Square              0
LifeSquare       2113
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1     4798
Helthcare_2         0
Shops_1             0
Shops_2             0
Price               0
dtype: int64

**Обрабатываем пропуски жилой площади**

В отдельном столбце помечаем данные с пропусками единичками

In [ ]:
# 1. общее для всех значение - 0
df['LifeSquare_nan'] = 0
df

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,LifeSquare_nan
0,11809,27,3.0,115.027311,NaN,10.0,4,10.0,2014,0.075424,B,B,11,3097,0,NaN,0,0,B,305018.871089,0
1,3013,22,1.0,39.832524,23.169223,8.0,7,8.0,1966,0.118537,B,B,30,6207,1,1183.0,1,0,B,177734.553407,0
2,8215,1,3.0,78.342215,47.671972,10.0,2,17.0,1988,0.025609,B,B,33,5261,0,240.0,3,1,B,282078.720850,0
3,2352,1,1.0,40.409907,NaN,1.0,10,22.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B,168106.007630,0
4,13866,94,2.0,64.285067,38.562517,9.0,16,16.0,1972,0.282798,B,B,33,8667,2,NaN,0,6,B,343995.102962,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1260,61,2.0,49.090728,33.272626,6.0,3,12.0,1981,0.300323,B,B,52,10311,6,NaN,1,9,B,119367.455796,0
9996,16265,27,2.0,64.307684,37.038420,9.0,13,0.0,1977,0.072158,B,B,2,629,1,NaN,0,0,A,199715.148807,0
9997,2795,178,1.0,29.648057,16.555363,5.0,3,5.0,1958,0.460556,B,B,20,4386,14,NaN,1,5,B,165953.912580,0
9998,14561,21,1.0,32.330292,22.326870,5.0,3,9.0,1969,0.194489,B,B,47,8004,3,125.0,3,5,B,171842.411855,0


In [ ]:
# 2. выбираем пустые зачения и помечаем их 1
df.loc[df['LifeSquare'].isna(), 'LifeSquare_nan'] = 1
df['LifeSquare_nan'].value_counts()

0    7887
1    2113
Name: LifeSquare_nan, dtype: int64

Заменяем пустые значения жилой площади значениями общей площади

In [ ]:
df['LifeSquare'] = df['LifeSquare'].fillna(df['Square'])
df['LifeSquare'].isna().sum()

0

In [ ]:
df[df['LifeSquare_nan'] == 1].head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,LifeSquare_nan
0,11809,27,3.0,115.027311,115.027311,10.0,4,10.0,2014,0.075424,B,B,11,3097,0,NaN,0,0,B,305018.871089,1
3,2352,1,1.0,40.409907,40.409907,1.0,10,22.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B,168106.007630,1
18,4581,1,1.0,47.582480,47.582480,1.0,16,16.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B,128803.593810,1
20,8768,63,1.0,50.974891,50.974891,1.0,7,16.0,2018,0.238617,B,B,26,3889,6,705.0,3,6,B,260328.413376,1
24,4717,27,1.0,44.203947,44.203947,0.0,2,0.0,1977,0.072158,B,B,2,629,1,NaN,0,0,A,135466.559958,1


**Обрабатываем пропуски Healthcare1**

In [ ]:
df['Healthcare_1'].describe()

count    5202.000000
mean     1142.904460
std      1021.517264
min         0.000000
25%       350.000000
50%       900.000000
75%      1548.000000
max      4849.000000
Name: Healthcare_1, dtype: float64

In [ ]:
# будем заменять на медианное значение
# 1. сделаем метки
df['Healthcare_1_nan'] = 0
df.loc[df['Healthcare_1'].isna(), 'Healthcare_1_nan'] = 1
df.loc[df['Healthcare_1_nan'] == 1].head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,LifeSquare_nan,Healthcare_1_nan
0,11809,27,3.0,115.027311,115.027311,10.0,4,10.0,2014,0.075424,B,B,11,3097,0,NaN,0,0,B,305018.871089,1,1
3,2352,1,1.0,40.409907,40.409907,1.0,10,22.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B,168106.007630,1,1
4,13866,94,2.0,64.285067,38.562517,9.0,16,16.0,1972,0.282798,B,B,33,8667,2,NaN,0,6,B,343995.102962,0,1
8,9951,1,1.0,46.887892,44.628132,1.0,12,20.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B,160400.401732,0,1
11,16755,88,2.0,53.975144,34.153584,8.0,2,12.0,1978,0.127376,B,B,43,8429,3,NaN,3,9,B,253124.005332,0,1


In [ ]:
# 2. рассчитаем медиану
median_health = df['Healthcare_1'].median()

In [ ]:
# 3. Заменяем пропуски на медианное значение
df['Healthcare_1'].fillna(median_health, inplace=True)
df['Healthcare_1'].isna().sum()

0

In [ ]:
df[df['Healthcare_1_nan'] == 1].head()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,LifeSquare_nan,Healthcare_1_nan
0,11809,27,3.0,115.027311,115.027311,10.0,4,10.0,2014,0.075424,B,B,11,3097,0,900.0,0,0,B,305018.871089,1,1
3,2352,1,1.0,40.409907,40.409907,1.0,10,22.0,1977,0.007122,B,B,1,264,0,900.0,0,1,B,168106.007630,1,1
4,13866,94,2.0,64.285067,38.562517,9.0,16,16.0,1972,0.282798,B,B,33,8667,2,900.0,0,6,B,343995.102962,0,1
8,9951,1,1.0,46.887892,44.628132,1.0,12,20.0,1977,0.007122,B,B,1,264,0,900.0,0,1,B,160400.401732,0,1
11,16755,88,2.0,53.975144,34.153584,8.0,2,12.0,1978,0.127376,B,B,43,8429,3,900.0,3,9,B,253124.005332,0,1


In [ ]:
df.isna().sum()

Id                  0
DistrictId          0
Rooms               0
Square              0
LifeSquare          0
KitchenSquare       0
Floor               0
HouseFloor          0
HouseYear           0
Ecology_1           0
Ecology_2           0
Ecology_3           0
Social_1            0
Social_2            0
Social_3            0
Healthcare_1        0
Helthcare_2         0
Shops_1             0
Shops_2             0
Price               0
LifeSquare_nan      0
Healthcare_1_nan    0
dtype: int64

#Обработка выбросов

In [ ]:
df.describe()

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price,LifeSquare_nan,Healthcare_1_nan
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,50.400800,1.890500,56.315775,42.091874,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1026.358900,1.319500,4.231300,214138.857399,0.211300,0.479800
std,43.587592,0.839512,21.058732,77.880874,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,746.662828,1.493601,4.806341,92872.293865,0.408251,0.499617
min,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028,0.000000,0.000000
25%,20.000000,1.000000,41.774881,25.506959,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,830.000000,0.000000,1.000000,153872.633942,0.000000,0.000000
50%,36.000000,2.000000,52.513310,37.562685,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879,0.000000,0.000000
75%,75.000000,2.000000,65.900625,50.098049,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,990.000000,2.000000,6.000000,249135.462171,0.000000,1.000000
max,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570,1.000000,1.000000


**Жилая площадь больше общей площади**

In [ ]:
df.loc[df['Square'] < df['LifeSquare']]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,LifeSquare_nan,Healthcare_1_nan
33,847,74,2.0,73.314975,74.131261,1.0,7,6.0,1977,0.075779,B,B,6,1437,3,900.0,0,2,B,254005.381863,0,1
64,3992,1,3.0,87.200625,87.730225,0.0,21,22.0,1977,0.007122,B,B,1,264,0,900.0,0,1,B,300180.365669,0,1
77,11771,45,1.0,38.467276,39.624493,1.0,11,9.0,1977,0.195781,B,B,23,5212,6,900.0,3,2,B,247440.530601,0,1
100,13343,74,2.0,79.937665,82.931595,1.0,8,21.0,1977,0.075779,B,B,6,1437,3,900.0,0,2,B,226808.747168,0,1
111,13875,20,1.0,28.438794,32.514021,6.0,6,10.0,2014,0.019509,B,B,37,7687,11,176.0,5,5,B,199248.114376,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9960,16258,48,3.0,96.056784,98.152802,1.0,15,1.0,2017,0.041125,B,B,46,9515,5,900.0,1,10,B,524365.550705,0,1
9962,14175,14,1.0,42.029175,44.682167,10.0,4,4.0,2018,0.238617,B,B,26,3889,6,705.0,3,6,B,87556.385617,0,0
9979,2201,6,1.0,40.043234,41.984322,1.0,3,17.0,2014,0.243205,B,B,5,1564,0,540.0,0,0,B,134888.083598,0,0
9987,11665,1,2.0,62.690057,64.602939,10.0,19,22.0,2018,0.007122,B,B,1,264,0,900.0,0,1,B,203908.192698,0,1


In [ ]:
# помечаем такие ячейки
df['LifeSquare_toobig'] = 0
df.loc[df['LifeSquare'] > df['Square'], 'LifeSquare_toobig'] = 1

In [ ]:
# заменяем значения жилой площади на значения общей площади
df.loc[df['LifeSquare'] > df['Square'], 'LifeSquare'] = df['Square']

In [ ]:
df.loc[df['LifeSquare_toobig'] == 1]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,LifeSquare_nan,Healthcare_1_nan,LifeSquare_toobig
33,847,74,2.0,73.314975,73.314975,1.0,7,6.0,1977,0.075779,B,B,6,1437,3,900.0,0,2,B,254005.381863,0,1,1
64,3992,1,3.0,87.200625,87.200625,0.0,21,22.0,1977,0.007122,B,B,1,264,0,900.0,0,1,B,300180.365669,0,1,1
77,11771,45,1.0,38.467276,38.467276,1.0,11,9.0,1977,0.195781,B,B,23,5212,6,900.0,3,2,B,247440.530601,0,1,1
100,13343,74,2.0,79.937665,79.937665,1.0,8,21.0,1977,0.075779,B,B,6,1437,3,900.0,0,2,B,226808.747168,0,1,1
111,13875,20,1.0,28.438794,28.438794,6.0,6,10.0,2014,0.019509,B,B,37,7687,11,176.0,5,5,B,199248.114376,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9960,16258,48,3.0,96.056784,96.056784,1.0,15,1.0,2017,0.041125,B,B,46,9515,5,900.0,1,10,B,524365.550705,0,1,1
9962,14175,14,1.0,42.029175,42.029175,10.0,4,4.0,2018,0.238617,B,B,26,3889,6,705.0,3,6,B,87556.385617,0,0,1
9979,2201,6,1.0,40.043234,40.043234,1.0,3,17.0,2014,0.243205,B,B,5,1564,0,540.0,0,0,B,134888.083598,0,0,1
9987,11665,1,2.0,62.690057,62.690057,10.0,19,22.0,2018,0.007122,B,B,1,264,0,900.0,0,1,B,203908.192698,0,1,1


**Количество этажей в доме = 0**

In [ ]:
df.loc[df['HouseFloor'] == 0]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,LifeSquare_nan,Healthcare_1_nan,LifeSquare_toobig
24,4717,27,1.0,44.203947,44.203947,0.0,2,0.0,1977,0.072158,B,B,2,629,1,900.0,0,0,A,135466.559958,1,1,0
152,12947,53,2.0,56.023342,56.023342,0.0,14,0.0,1977,0.049637,B,B,34,7759,0,229.0,1,3,B,247074.564832,1,0,0
159,13146,74,1.0,46.449711,46.449711,0.0,6,0.0,1977,0.075779,B,B,6,1437,3,900.0,0,2,B,168705.450148,1,1,0
186,12174,62,1.0,44.954880,44.954880,0.0,8,0.0,1977,0.072158,B,B,2,629,1,900.0,0,0,A,133773.301722,1,1,0
229,15661,45,3.0,81.094922,81.094922,0.0,15,0.0,1977,0.195781,B,B,23,5212,6,900.0,3,2,B,442472.456032,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9889,629,27,1.0,42.936724,42.936724,0.0,2,0.0,1977,0.011654,B,B,4,915,0,900.0,0,0,B,129674.841139,1,1,0
9931,15686,1,1.0,47.402412,47.402412,0.0,14,0.0,1977,0.007122,B,B,1,264,0,900.0,0,1,B,174740.301224,1,1,0
9940,4711,6,1.0,41.224118,41.224118,0.0,8,0.0,1977,0.243205,B,B,5,1564,0,540.0,0,0,B,125612.639104,1,0,0
9950,5383,48,3.0,98.994909,98.994909,0.0,3,0.0,1977,0.041125,B,B,46,9515,5,900.0,1,10,B,418817.004446,1,1,0


In [ ]:
# помечаем такие объекты
df['HouseFloor_null'] = 0
df.loc[df['HouseFloor'] == 0, 'HouseFloor_null'] = 1

In [ ]:
# заменяем на значения этажности квартиры
df.loc[df['HouseFloor'] == 0, 'HouseFloor'] = df['Floor']

In [ ]:
df.loc[df['HouseFloor_null'] == 1]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,LifeSquare_nan,Healthcare_1_nan,LifeSquare_toobig,HouseFloor_null
24,4717,27,1.0,44.203947,44.203947,0.0,2,2.0,1977,0.072158,B,B,2,629,1,900.0,0,0,A,135466.559958,1,1,0,1
152,12947,53,2.0,56.023342,56.023342,0.0,14,14.0,1977,0.049637,B,B,34,7759,0,229.0,1,3,B,247074.564832,1,0,0,1
159,13146,74,1.0,46.449711,46.449711,0.0,6,6.0,1977,0.075779,B,B,6,1437,3,900.0,0,2,B,168705.450148,1,1,0,1
186,12174,62,1.0,44.954880,44.954880,0.0,8,8.0,1977,0.072158,B,B,2,629,1,900.0,0,0,A,133773.301722,1,1,0,1
229,15661,45,3.0,81.094922,81.094922,0.0,15,15.0,1977,0.195781,B,B,23,5212,6,900.0,3,2,B,442472.456032,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9889,629,27,1.0,42.936724,42.936724,0.0,2,2.0,1977,0.011654,B,B,4,915,0,900.0,0,0,B,129674.841139,1,1,0,1
9931,15686,1,1.0,47.402412,47.402412,0.0,14,14.0,1977,0.007122,B,B,1,264,0,900.0,0,1,B,174740.301224,1,1,0,1
9940,4711,6,1.0,41.224118,41.224118,0.0,8,8.0,1977,0.243205,B,B,5,1564,0,540.0,0,0,B,125612.639104,1,0,0,1
9950,5383,48,3.0,98.994909,98.994909,0.0,3,3.0,1977,0.041125,B,B,46,9515,5,900.0,1,10,B,418817.004446,1,1,0,1


**Неадекватная площадь кухни (до 2014 мкв)**

In [ ]:
df['KitchenSquare'].describe()

count    10000.000000
mean         6.273300
std         28.560917
min          0.000000
25%          1.000000
50%          6.000000
75%          9.000000
max       2014.000000
Name: KitchenSquare, dtype: float64

In [ ]:
# поиск с помощью квантилей
KitchenSquare_max = np.quantile(df['KitchenSquare'], q=0.995)
KitchenSquare_max

20.0

In [ ]:
KitchenSquare_min = np.quantile(df['KitchenSquare'], q=0.05)
KitchenSquare_min
# ничего не делаем, так как кухни может не быть как отдельного помещения

0.0

In [ ]:
condition = df['KitchenSquare'] > KitchenSquare_max
df[condition]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,LifeSquare_nan,Healthcare_1_nan,LifeSquare_toobig,HouseFloor_null
277,13559,79,1.0,96.838013,49.140960,22.0,19,24.0,2016,0.050756,B,B,24,5469,1,145.0,0,1,B,348962.861294,0,0,0,0
286,2737,27,3.0,123.430072,123.430072,123.0,5,10.0,2015,0.017647,B,B,2,469,0,900.0,0,0,B,234194.837047,0,1,1,0
456,10546,62,1.0,46.013187,43.417043,43.0,4,17.0,2019,0.072158,B,B,2,629,1,900.0,0,0,A,113909.099428,0,1,0,0
511,8685,62,5.0,124.933289,81.122369,30.0,5,3.0,2017,0.072158,B,B,2,629,1,900.0,0,0,A,271919.866048,0,1,0,0
682,16593,52,2.0,64.859242,64.859242,62.0,15,17.0,1977,0.371149,B,B,34,7065,1,750.0,2,5,B,185993.474671,1,0,0,0
755,15781,10,1.0,35.813564,19.953992,35.0,6,14.0,1971,0.282798,B,B,33,8667,2,900.0,0,6,B,164306.314182,0,1,0,0
1614,3267,58,3.0,138.980817,138.004600,43.0,2,2.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,169528.896664,0,0,0,0
1728,16395,2,3.0,79.722243,44.731219,72.0,12,16.0,1987,0.130618,B,B,39,10418,9,900.0,1,9,B,370148.625285,0,0,0,0
1776,16568,27,4.0,200.334539,200.334539,25.0,1,2.0,2013,0.041116,B,B,53,14892,4,900.0,1,4,B,528560.506016,0,1,1,0
1906,5548,86,5.0,275.645284,233.949309,26.0,12,37.0,2011,0.161976,B,A,31,7010,5,4508.0,3,7,B,455264.882666,0,0,0,0


In [ ]:
# заменяем признак на медиану
df.loc[condition, 'KitchenSquare'] = df['KitchenSquare'].median()
df['KitchenSquare'].describe()

count    10000.000000
mean         5.675700
std          3.874639
min          0.000000
25%          1.000000
50%          6.000000
75%          9.000000
max         20.000000
Name: KitchenSquare, dtype: float64

**Этаж квартиры больше, чем количество этажей в здании**

In [ ]:
condition_floor = df['HouseFloor'] < df['Floor']

In [ ]:
# помечаем такие объекты
df['Floor_more_then_House'] = 0
df.loc[condition_floor, 'Floor_more_then_House'] = 1
df.loc[condition_floor]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,LifeSquare_nan,Healthcare_1_nan,LifeSquare_toobig,HouseFloor_null,Floor_more_then_House
17,1190,8,2.0,49.396000,28.299600,5.0,6,5.0,1962,0.458174,B,A,26,4439,1,1130.0,1,5,B,186971.926566,0,0,0,0,1
19,367,45,1.0,42.367289,40.012529,1.0,16,14.0,2015,0.195781,B,B,23,5212,6,900.0,3,2,B,250466.980777,0,1,0,0,1
21,8471,20,3.0,75.611065,47.494621,8.0,11,9.0,1976,0.019509,B,B,37,7687,11,176.0,5,5,B,366916.246494,0,0,0,0,1
25,4521,6,2.0,65.409965,65.409965,1.0,18,1.0,1977,0.243205,B,B,5,1564,0,540.0,0,0,B,191205.272608,1,0,0,0,1
26,8553,88,3.0,83.262530,4.769174,1.0,13,1.0,1977,0.127376,B,B,43,8429,3,900.0,3,9,B,410883.892020,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9953,7659,57,1.0,38.485252,22.550608,8.0,19,17.0,1992,0.133215,B,B,49,11395,3,1406.0,3,4,A,183485.284995,0,0,0,0,1
9960,16258,48,3.0,96.056784,96.056784,1.0,15,1.0,2017,0.041125,B,B,46,9515,5,900.0,1,10,B,524365.550705,0,1,1,0,1
9968,9920,1,3.0,86.487550,86.487550,1.0,18,17.0,1977,0.007122,B,B,1,264,0,900.0,0,1,B,248248.935528,1,1,0,0,1
9970,13501,6,1.0,41.310416,19.117077,9.0,18,17.0,1977,0.243205,B,B,5,1564,0,540.0,0,0,B,114878.830713,0,0,0,0,1


In [ ]:
# Заменяем значения этажности дома на этаж квартиры
df.loc[condition_floor, 'HouseFloor'] = df['Floor']
df.loc[condition_floor]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price,LifeSquare_nan,Healthcare_1_nan,LifeSquare_toobig,HouseFloor_null,Floor_more_then_House
17,1190,8,2.0,49.396000,28.299600,5.0,6,6.0,1962,0.458174,B,A,26,4439,1,1130.0,1,5,B,186971.926566,0,0,0,0,1
19,367,45,1.0,42.367289,40.012529,1.0,16,16.0,2015,0.195781,B,B,23,5212,6,900.0,3,2,B,250466.980777,0,1,0,0,1
21,8471,20,3.0,75.611065,47.494621,8.0,11,11.0,1976,0.019509,B,B,37,7687,11,176.0,5,5,B,366916.246494,0,0,0,0,1
25,4521,6,2.0,65.409965,65.409965,1.0,18,18.0,1977,0.243205,B,B,5,1564,0,540.0,0,0,B,191205.272608,1,0,0,0,1
26,8553,88,3.0,83.262530,4.769174,1.0,13,13.0,1977,0.127376,B,B,43,8429,3,900.0,3,9,B,410883.892020,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9953,7659,57,1.0,38.485252,22.550608,8.0,19,19.0,1992,0.133215,B,B,49,11395,3,1406.0,3,4,A,183485.284995,0,0,0,0,1
9960,16258,48,3.0,96.056784,96.056784,1.0,15,15.0,2017,0.041125,B,B,46,9515,5,900.0,1,10,B,524365.550705,0,1,1,0,1
9968,9920,1,3.0,86.487550,86.487550,1.0,18,18.0,1977,0.007122,B,B,1,264,0,900.0,0,1,B,248248.935528,1,1,0,0,1
9970,13501,6,1.0,41.310416,19.117077,9.0,18,18.0,1977,0.243205,B,B,5,1564,0,540.0,0,0,B,114878.830713,0,0,0,0,1


# Новые признаки

Изменяем тип данных категорийных признаков

In [ ]:
df.dtypes

In [ ]:
df['Shops_2'].unique()

array(['B', 'A'], dtype=object)

In [ ]:
# заменяем значения A и B Shops_2 цифрами 1 и 2
df['Shops_2_int'] = 1
df.loc[df['Shops_2'] == 'B', 'Shops_2_int'] = 2

In [ ]:
df['Ecology_2'].unique()

array(['B', 'A'], dtype=object)

In [ ]:
# заменяем значения A и B Ecology_2 цифрами 1 и 2
df['Ecology_2_int'] = 1
df.loc[df['Ecology_2'] == 'B', 'Ecology_2_int'] = 2

In [ ]:
df['Ecology_3'].unique()

array(['B', 'A'], dtype=object)

In [ ]:
# заменяем значения A и B Ecology_3 цифрами 1 и 2
df['Ecology_3_int'] = 1
df.loc[df['Ecology_3'] == 'B', 'Ecology_3_int'] = 2

In [ ]:
df.dtypes

In [ ]:
# добавляем новый признак средней цены квартиры по году дома (для учета года постройки)
def code_mean(data, cat_feature, real_feature):
  return (data[cat_feature].map(data.groupby(cat_feature)[real_feature].mean()))

df['mean_year_price']

# **Класс для очистки**

In [ ]:
# класс для очистки датасета
class DataPipeLine:
  """ подготовка исходных данных """
  def __init__(self):
    self.Healthcare_1_median = None
    self.KitchenSquare_median = None
    self.KitchenSquare_max = None

  """ сохраняем статистики """
  def fit(self, df):
    self.Healthcare_1_median = df['Healthcare_1'].median()
    self.KitchenSquare_median = df['KitchenSquare'].median()
    self.KitchenSquare_max = np.quantile(df['KitchenSquare'], q=0.995)

  """ трансформация данных """    
  def transform(self, df):

    # 1. Обработка пропусков
    df['LifeSquare'] = df['LifeSquare'].fillna(df['Square'])
    df['Healthcare_1'] = df['Healthcare_1'].fillna(self.Healthcare_1_median)

    # 2. Корректировка значений

    # жилая площадь больше общей
    df.loc[df['LifeSquare'] > df['Square'], 'LifeSquare'] = df['Square']

    # количество этажей в доме = 0
    df.loc[df['HouseFloor'] == 0, 'HouseFloor'] = df['Floor']

    # этаж квартиры больше,чем этажность здания
    df.loc[df['HouseFloor'] < df['Floor'], 'HouseFloor'] = df['Floor']

    # 3. Выбросы

    # площадь кухни
    df.loc[df['KitchenSquare'] > self.KitchenSquare_max, 'KitchenSquare'] = self.KitchenSquare_median

    return df

In [ ]:
df = pd.read_csv(DATASET_PATH, sep=',')
pipe = DataPipeLine()
pipe.fit(df)
df = pipe.transform(df)
df.describe()

In [ ]:
df.to_csv(PREPARED_DATASET_PATH, index=False, encoding='utf-8')